<a href="https://colab.research.google.com/github/yemauricio/caramelo/blob/main/etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejecución Manual ETL caramelo

### Clonar repositorio

In [6]:
! git clone https://github.com/yemauricio/caramelo.git

Cloning into 'caramelo'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
cd caramelo/

In [ ]:
!pwd

/content/caramelo


In [ ]:
%run -i 'etl.py'

In [ ]:
cd resultado/


/content/caramelo/resultado


In [ ]:
ls -la

total 380
drwxr-xr-x 2 root root   4096 Jan 25 22:57 ./
drwxr-xr-x 7 root root   4096 Jan 25 22:48 ../
-rw-r--r-- 1 root root    989 Jan 25 22:57 case_actuaciones.txt
-rw-r--r-- 1 root root   2243 Jan 25 22:57 case_preguntas.txt
-rw-r--r-- 1 root root 264415 Jan 25 22:57 correlacion_variables.csv
-rw-r--r-- 1 root root  86883 Jan 25 22:57 id_puntajes.csv
-rw-r--r-- 1 root root  13734 Jan 25 22:55 resultado.csv
